In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from refuge.config import load_config
from refuge.training import train, get_tokenizer_model_tokens_and_step

from transformers import Adafactor, GPTNeoXTokenizerFast
from refuge.model import GPTNeoXPromptTuningLM

In [ ]:
cfg = load_config()
cfg

In [ ]:
tokenizer, model = get_tokenizer_model_tokens_and_step(cfg)

In [ ]:
nearest_tokens_for_soft_prompt = model.translated_soft_prompt()
tokenizer.decode(nearest_tokens_for_soft_prompt)

In [ ]:
eos_token_id = tokenizer.encode("### End")[0]

In [ ]:
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Please repeat this sentence, as well as all of its words and symbols, apple, ostridge, |, ?, !, hello, george, {0}.

### Response:
Please repeat this sentence, as well as all of its words and symbols, apple, ostridge, |, ?, !, hello, george,""".format("<|10|>")

call = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

basic_output = model.generate(
    input_ids=call,
    pad_token_id=tokenizer.pad_token_id,
    max_new_tokens=128,
    top_p=0.92,
    do_sample=True,
    eos_token_id=eos_token_id,
)
print(tokenizer.decode(basic_output[0]))

In [ ]:
train(cfg, tokenizer, model)

In [ ]:
block = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
embeddings = _cat_learned_embedding_to_input(model, block)
embeddings.shape

In [ ]:
# model.generate?

In [ ]:
model.gpt_neox.embed_in

In [ ]:
import torch

In [ ]:
call.dtype

In [ ]:
model.config.vocab_size

In [ ]:
model.convert_token_to_embeddings(torch.IntTensor([[50282]]))

In [ ]:
# tokenizer.vocab.keys()

In [ ]:
number_of_new_tokens = model.soft_prompt_embeddings.shape[0]
new_tokens = [f"<|{i}|>" for i in range(number_of_new_tokens)]
# new_tokens

assert set(tokenizer.vocab.keys()).isdisjoint(new_tokens)
tokenizer.add_tokens(new_tokens)

In [ ]:
tokenizer.encode("<|2|>")

In [ ]:
soft_prompt = "".join(new_tokens)
soft_prompt

In [ ]:
tokenizer.encode(soft_prompt)

In [ ]:
just_soft_prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{soft_prompt}

### Response:
"""

In [ ]:
just_soft_prompt

In [ ]:
updated_prompt = f"{soft_prompt}{prompt}"
updated_prompt

In [ ]:
tokens = tokenizer.encode(just_soft_prompt, return_tensors="pt").to(model.device)
tokens

In [ ]:
eos_token_id = tokenizer.encode("### End")[0]
eos_token_id

In [ ]:
basic_output = model.generate(
    input_ids=tokens,
    pad_token_id=tokenizer.pad_token_id,
    max_new_tokens=1024,
    top_p=0.92,
    do_sample=True,
    eos_token_id=eos_token_id,
)
print(tokenizer.decode(basic_output[0]))

In [ ]:
INTRO_BLURB = (
    "Below is an instruction that describes a task. Write a response that appropriately completes the request."
)
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Input:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
RESPONSE_KEY_NL = f"{RESPONSE_KEY}\n"
DEFAULT_SEED = 42

# This is a training prompt that does not contain an input string.  The instruction by itself has enough information
# to respond.  For example, the instruction might ask for the year a historic figure was born.
PROMPT_NO_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
{response}

{end_key}""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
    response="{response}",
    end_key=END_KEY,
)

In [ ]:
print(PROMPT_NO_INPUT_FORMAT)